In [7]:
ref='nb92-' #Note to matt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import matplotlib
plt.style.use('seaborn-whitegrid')
matplotlib.rcParams.update({
    'font.family': 'serif',
    'axes.titlesize':16,
    'axes.labelsize':16,
    'axes.xmargin':0.1,
    'axes.ymargin':0.1,
    'legend.fontsize':16,
    'xtick.labelsize' : 16,
    'ytick.labelsize' : 16,
    'lines.markersize': 10,
    'lines.linewidth' : 3,
    'font.size': 16
})


import pickle 
import tempfile
import os


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
import mlflow.tensorflow

RANDOM_STATE=24


# Define MLP Model

In [2]:
def make_mlp(inp_dim, run): #Functional API #Default initializer for all layers is Xavier (aka glorot_uniform)
    inputs = keras.Input(shape=(inp_dim,), name='input') 
    x = Dense(run['nodes'][0], activation=run['act'][0])(inputs)
    x = Dropout(run['dropout'][0])(x)
    for i in range(int(len(run['nodes'])-1)):
        x = Dense(run['nodes'][i+1], activation=run['act'][i+1])(x)
        x=Dropout(run['dropout'][i+1])(x)
    outputs = Dense(1, activation='linear')(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name='model')
    return model

# Import Data

In [3]:
def make_data(data_key):
    if data_key=='xyz_raw':
        data_file_path="../data/processed/mlp_dataset_raw_xyz.csv"
        dataset=pd.read_csv(data_file_path,index_col=False )
        X=dataset.iloc[:,0:300].values
        y=dataset.iloc[:,300].values
    elif data_key=='xyz_interp':
        data_file_path="../data/processed/mlp_dataset_interp_xyz.csv"
        dataset=pd.read_csv(data_file_path,index_col=False )
        X=dataset.iloc[:,0:300].values
        y=dataset.iloc[:,300].values
    elif data_key=='rms_raw':
        data_file_path="../data/processed/mlp_dataset_raw_rms.csv"
        dataset=pd.read_csv(data_file_path,index_col=False )
        X=dataset.iloc[:,0:100].values
        y=dataset.iloc[:,100].values
    elif data_key == 'rms_interp':
        data_file_path="../data/processed/mlp_dataset_interp_rms.csv"
        dataset=pd.read_csv(data_file_path,index_col=False )
        X=dataset.iloc[:,0:100].values
        y=dataset.iloc[:,100].values
    #Split training/testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
    return X_train, X_test,y_train, y_test, data_file_path

# Import Hyperparameters

In [4]:
# Import Hyperparameters
from mlp_params_9 import grid

No. of combinations: 32


# Train and Test

In [5]:
mlflow.set_experiment("Gait Speed Prediction")

for i,run in enumerate(grid):
    with mlflow.start_run(run_name=str(run['id'])):
        mlflow.set_tag('Model', run['model'])
        mlflow.set_tag('Dataset', run['dataset'])
        mlflow.set_tag('Data splitting Random State', RANDOM_STATE)
        scaler=run['scaler']
        mlflow.set_tag('Scaler', str(scaler.get_params()))
        X_train, X_test,y_train, y_test, data_file_path=make_data(run['dataset'])
        mlflow.set_tag('Data File Path', data_file_path)
        # mlflow.log_artifact(data_file_path) #Uncomment this if dataset needs ot be saved along with model
        sX_train=scaler.fit_transform(X_train)
        with tempfile.TemporaryDirectory() as tmpdir:
            pickle.dump(scaler, open(os.path.join(tmpdir,'scaler.pkl'), 'wb'))
            mlflow.log_artifact(os.path.join(tmpdir,'scaler.pkl'))
        model=make_mlp(X_train.shape[1], run)
        model.compile(loss=run['loss'] , optimizer="adam", metrics=run['metrics'])
        mlflow.tensorflow.autolog() #This will log params passed in model.fit
        history = model.fit(sX_train,y_train, batch_size=run['batch_size'], epochs=run['epochs'], verbose=1, validation_split=0.1)
        # Testing
        sX_test=scaler.transform(X_test)
        score = model.evaluate(sX_test, y_test,  verbose=1)
        mlflow.log_metrics({'test_loss': score[0], 'test_RMSE': np.sqrt(score[1])})
    

2022-01-11 16:37:59.626953: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-11 16:37:59.627101: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-11 16:37:59.627147: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-01-11 16:37:59.627185: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-01-11 16:37:59.627222: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

Epoch 1/80
919/919 [==============================] - 3s 3ms/step - loss: 0.1986 - mean_squared_error: 0.1986 - val_loss: 0.1235 - val_mean_squared_error: 0.1235
Epoch 2/80
919/919 [==============================] - 2s 2ms/step - loss: 0.0881 - mean_squared_error: 0.0881 - val_loss: 0.0953 - val_mean_squared_error: 0.0953
Epoch 3/80
919/919 [==============================] - 2s 2ms/step - loss: 0.0714 - mean_squared_error: 0.0714 - val_loss: 0.0707 - val_mean_squared_error: 0.0707
Epoch 4/80
919/919 [==============================] - 2s 2ms/step - loss: 0.0638 - mean_squared_error: 0.0638 - val_loss: 0.0643 - val_mean_squared_error: 0.0643
Epoch 5/80
919/919 [==============================] - 2s 2ms/step - loss: 0.0603 - mean_squared_error: 0.0603 - val_loss: 0.0658 - val_mean_squared_error: 0.0658
Epoch 6/80
919/919 [==============================] - 2s 2ms/step - loss: 0.0594 - mean_squared_error: 0.0594 - val_loss: 0.0671 - val_mean_squared_error: 0.0671
Epoch 7/80
919/919 [========

2022-01-11 16:40:07.921428: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpfaalzhpu/model/data/model/assets
128/128 [==============================] - 0s 991us/step - loss: 0.0682 - mean_squared_error: 0.0682
Epoch 1/80
919/919 [==============================] - 3s 2ms/step - loss: 0.1145 - mean_squared_error: 0.1145 - val_loss: 0.0699 - val_mean_squared_error: 0.0699
Epoch 2/80
919/919 [==============================] - 2s 2ms/step - loss: 0.0632 - mean_squared_error: 0.0632 - val_loss: 0.0629 - val_mean_squared_error: 0.0629
Epoch 3/80
919/919 [==============================] - 2s 2ms/step - loss: 0.0588 - mean_squared_error: 0.0588 - val_loss: 0.0607 - val_mean_squared_error: 0.0607
Epoch 4/80
919/919 [==============================] - 2s 2ms/step - loss: 0.0564 - mean_squared_error: 0.0564 - val_loss: 0.0619 - val_mean_squared_error: 0.0619
Epoch 5/80
919/919 [==============================] - 2s 2ms/step - loss: 0.0555 - mean_squared_error: 0.0555 - val_loss: 0.0585 - val_mean_squared_error: 0.0585
Epoch 6/80
91

In [6]:
# #Predicting based on selected run (to be used in production)

# #Retrieve Scaler
# client = mlflow.tracking.MlflowClient() 
# local_dir = "/tmp/artifact_downloads" 
# if not os.path.exists(local_dir): 
# 	os.mkdir(local_dir) 

# local_path = client.download_artifacts('3329b6cce2fd426cad7194f9da526103', '', local_dir) 
# file = open('/tmp/artifact_downloads/scaler.pkl', 'rb')
# scaler=pickle.load(file)

# #Retrieve Model
# logged_model = 'runs:/3329b6cce2fd426cad7194f9da526103/model'
# # Load model as a PyFuncModel.
# loaded_model = mlflow.pyfunc.load_model(logged_model)

# #Predict 
# sX_test=scaler.transform(X_test)
# loaded_model.predict(sX_test)